<a href="https://www.kaggle.com/code/mattspotify2023/dogs-vs-cats-pytorch-resnet50-98-acc?scriptVersionId=196435716" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Importing all the packages
import numpy as np
import pandas as pd
import torch
import random
import os,shutil
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
from torch import optim
from torch import nn
import cv2
from glob import glob
import matplotlib.pyplot as plt
from torchvision.datasets import DatasetFolder
from torchvision.datasets import ImageFolder
from torchvision import transforms,models,datasets



In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

In [ ]:

import zipfile

with zipfile.ZipFile("/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip","r") as z:
    z.extractall(".")
    
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip","r") as z:
    z.extractall(".")

!ls /kaggle/working

In [ ]:
 for file in os.listdir():
    if os.path.isdir(file):
        print(file)

In [ ]:

os.chdir(r'/kaggle/working/train')
!mkdir train valid
os.chdir(r'/kaggle/working/train/train')
!mkdir cats dogs
os.chdir(r'/kaggle/working/train/valid')
!mkdir cats dogs



In [ ]:
#directories name
original_dir = '/kaggle/working/train'
train_dir = '/kaggle/working/train/train'
valid_dir = '/kaggle/working/train/valid'
cats_train = '/kaggle/working/train/train/cats'
dogs_train = '/kaggle/working/train/train/dogs'
cats_valid = '/kaggle/working/train/valid/cats'
dogs_valid = '/kaggle/working/train/valid/dogs'


In [ ]:
#Moving folders
import shutil
import os
dogs = 0
cats = 0
for file in os.listdir(original_dir):
    if file.startswith('dog.'):
        if dogs <=11250:
            shutil.move(os.path.join(original_dir,file),os.path.join(dogs_train,file))
        else:
            shutil.move(os.path.join(original_dir,file),os.path.join(dogs_valid,file))
        dogs+=1
    elif file.startswith('cat.'):
        if cats <= 11250:
            shutil.move(os.path.join(original_dir,file),os.path.join(cats_train,file))
        else:
            shutil.move(os.path.join(original_dir,file),os.path.join(cats_valid,file))
        cats+=1

print(dogs,cats)

In [ ]:
!ls /kaggle/working/train/train


In [ ]:
transforms = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])

In [ ]:
from torchvision.datasets import ImageFolder

In [ ]:
#Dataset class
train_dataset = ImageFolder(root = train_dir,transform=transforms)
valid_dataset = ImageFolder(root = valid_dir,transform=transforms)

In [ ]:
#Visualize a random image
import random
index = random.randint(0,len(train_dataset)-1)
image,label = train_dataset[index]
plt.imshow(image.numpy().transpose(1,2,0))
plt.title(label)

In [ ]:
#Dataloader
train_dl = DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers = 4)
valid_dl = DataLoader(valid_dataset,batch_size=32,shuffle=False)


In [ ]:
#Defining model

class mynet(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.convnet = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            nn.Conv2d(128,128,3,1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(128 * 12 * 12,2)
        )
    
    def forward(self,x):
        x = self.convnet(x)
        return x
    

model = mynet().to(device)

In [ ]:
pip install torchsummary

In [ ]:
from torchsummary import summary

summary(model,input_size=(3,224,224))

In [ ]:
from torch.optim import SGD,Adam
opt = SGD(model.parameters(),lr = 1e-03)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# #The Main Loop
# epochs = 2
# train_losses,test_losses = [],[]
# train_accs,test_accs = [],[]
# for epoch in range(epochs):
#     train_loss = 0.0
#     correct = 0
#     train_acc = 0.0
#     #training loop

#     for batch,(x,y) in enumerate(train_dl):
#         model.train()
#         x,y = x.to(device),y.to(device)
#         pred = model(x)
#         loss = loss_fn(pred,y)
#         loss.backward()
#         opt.step()
#         opt.zero_grad()
#         train_loss+=loss.item()
#         y_pred_class = torch.argmax(torch.softmax(pred,dim=1),dim=1)
#         train_acc += (y_pred_class == y).sum().item()/len(pred)
#     avg_train_loss = train_loss/len(train_dl)
#     avg_train_acc = train_acc/len(train_dl)
#     train_losses.append(avg_train_loss)
#     train_accs.append(avg_train_acc)
#     print(f'Epoch: {epoch} train loss: {avg_train_loss} train acc: {avg_train_acc}')
    
#     #Test loop
#     model.eval()
#     test_loss = 0.0 
#     test_correct = 0
#     test_acc = 0.0
#     with torch.no_grad():
#         for batch,(x,y) in enumerate(valid_dl):
#             x,y = x.to(device),y.to(device)
#             pred = model(x)
#             loss = loss_fn(pred.squeeze(0),y)
#             test_loss += loss.item()
#             y_pred_class_test = torch.argmax(torch.softmax(pred, dim=1), dim=1)
#             test_acc += (y_pred_class_test == y).sum().item()/len(pred)
#         avg_test_loss = test_loss/len(valid_dl)
#         avg_test_acc = test_acc/len(valid_dl)
#         test_accs.append(avg_test_acc)
#         test_losses.append(avg_test_loss)
#         print(f'Epoch: {epoch} test loss: {avg_test_loss} test acc: {avg_test_acc}')

In [ ]:
# test_dir = r'/kaggle/working/test'
# from torchvision import transforms,models,datasets
# def transform_image(image):
#     image_path = r'{}'.format(image)
#     custom_image = torchvision.io.read_image(str(image_path)).type(torch.float32)
#     custom_image /= 255
#     custom_trans = transforms.Compose([transforms.Resize((224,224))])
#     custom_image_transformed = custom_trans(custom_image) 

#     return custom_image_transformed

# def predict_image(image,model):
#     model.eval()
#     with torch.no_grad():
#         custom_pred = model(image.unsqueeze(dim=0).to(device))
#         probs = torch.nn.functional.softmax(custom_pred[0],dim=0)
#         predicted_class = torch.argmax(probs).item()
#     return predicted_class

# predictions = []
# image_names = []
# ids = []
# for image in os.listdir(test_dir):
#     image_path = os.path.join(test_dir,image) #Full path
#     #adding name 
#     ids.append(image.split(".")[0])
#     image_names.append(image)
#     prediction = predict_image(transform_image(image_path),model)
#     predictions.append(prediction)
# os.chdir(r'/kaggle/working/')
# import pandas as pd    
# df = pd.DataFrame({"id":ids,"label":predictions})
# df.to_csv('submission.csv',index=False)

In [ ]:
# import sys
# sys.exit()


In [ ]:
# tl_model = models.resnet18(pretrained = True)
# #Freeze the parameters
# for param in tl_model.parameters():
#     param.requires_grad = False
    
# #Modify the last FCN
# num_classes = 2
# # modify the output size of pooling
# tl_model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
# input_tolinear = tl_model.fc.in_features
# tl_model.fc =nn.Linear(input_tolinear,num_classes)
# tl_model.to(device='cuda')


In [ ]:
# from torchsummary import summary

# summary(tl_model,input_size=(3,224,224))

In [ ]:
# # %%
# from torch.optim import Adam,SGD

# loss_fn = nn.CrossEntropyLoss()

# opt = SGD(tl_model.parameters(),lr = 1e-03)

# # %%
# epochs = 10
# train_losses,test_losses = [],[]
# train_accs,test_accs = [],[]
# for epoch in range(epochs):
#     train_loss = 0.0
#     correct = 0
#     train_acc = 0.0
#     for batch,(x,y) in enumerate(train_dl):

#         tl_model.train()
#         x,y = x.to(device),y.to(device)
#         pred = tl_model(x)
#         loss = loss_fn(pred,y)
#         loss.backward()
#         opt.step()
#         opt.zero_grad()
#         train_loss += loss.item()
#         # correct += (pred.argmax(1)== y).float().sum()
#         y_pred_class = torch.argmax(torch.softmax(pred, dim=1), dim=1)
#         train_acc += (y_pred_class == y).sum().item()/len(pred)
#     avg_train_loss = train_loss/len(train_dl)
#     avg_train_acc = train_acc/len(train_dl)
#     train_losses.append(avg_train_loss)
#     train_accs.append(avg_train_acc)
#     print(f'Epoch: {epoch} train loss: {avg_train_loss} train acc: {avg_train_acc}')

#     #testloop
#     tl_model.eval()
#     test_loss = 0.0
#     test_correct = 0
#     test_acc = 0.0
#     with torch.no_grad():
#         for batch,(x,y) in enumerate(valid_dl):
#             x,y = x.to(device),y.to(device)
#             pred = tl_model(x)
#             loss = loss_fn(pred.squeeze(0),y)
#             test_loss += loss.item()
#             y_pred_class_test = torch.argmax(torch.softmax(pred, dim=1), dim=1)
#             # print(y_pred_class_test,y)
#             test_acc += (y_pred_class_test == y).sum().item()/len(pred)

#         avg_test_loss = test_loss/len(valid_dl)
#         avg_test_acc = test_acc/len(valid_dl)
#         test_accs.append(avg_test_acc)
#         test_losses.append(avg_test_loss)
#         print(f'Epoch: {epoch} test loss: {avg_test_loss} test acc: {avg_test_acc}')

In [ ]:
# plt.plot(train_losses,label='train_loss')
# plt.plot(test_losses,label='test_loss')
# plt.legend()

RESNET50

In [ ]:
tl_model2 = models.resnet50(pretrained = True)
#Freeze the parameters
for param in tl_model2.parameters():
    param.requires_grad = False
    
#Modify the last FCN
num_classes = 2
# modify the output size of pooling
tl_model2.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
input_tolinear = tl_model2.fc.in_features
tl_model2.fc =nn.Linear(input_tolinear,num_classes)
tl_model2.to(device='cuda')


In [ ]:
pip install torchsummary

In [ ]:
from torchsummary import summary

summary(tl_model2,input_size=(3,224,224))

In [ ]:
# %%
from torch.optim import Adam,SGD

loss_fn = nn.CrossEntropyLoss()

opt = SGD(tl_model2.parameters(),lr = 1e-03)

# %%
epochs = 20
train_losses,test_losses = [],[]
train_accs,test_accs = [],[]
for epoch in range(epochs):
    train_loss = 0.0
    correct = 0
    train_acc = 0.0
    for batch,(x,y) in enumerate(train_dl):

        tl_model2.train()
        x,y = x.to(device),y.to(device)
        pred = tl_model2(x)
        loss = loss_fn(pred,y)
        loss.backward()
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()
        # correct += (pred.argmax(1)== y).float().sum()
        y_pred_class = torch.argmax(torch.softmax(pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(pred)
    avg_train_loss = train_loss/len(train_dl)
    avg_train_acc = train_acc/len(train_dl)
    train_losses.append(avg_train_loss)
    train_accs.append(avg_train_acc)
    print(f'Epoch: {epoch} train loss: {avg_train_loss} train acc: {avg_train_acc}')

    #testloop
    tl_model2.eval()
    test_loss = 0.0
    test_correct = 0
    test_acc = 0.0
    with torch.no_grad():
        for batch,(x,y) in enumerate(valid_dl):
            x,y = x.to(device),y.to(device)
            pred = tl_model2(x)
            loss = loss_fn(pred.squeeze(0),y)
            test_loss += loss.item()
            y_pred_class_test = torch.argmax(torch.softmax(pred, dim=1), dim=1)
            # print(y_pred_class_test,y)
            test_acc += (y_pred_class_test == y).sum().item()/len(pred)

        avg_test_loss = test_loss/len(valid_dl)
        avg_test_acc = test_acc/len(valid_dl)
        test_accs.append(avg_test_acc)
        test_losses.append(avg_test_loss)
        print(f'Epoch: {epoch} test loss: {avg_test_loss} test acc: {avg_test_acc}')

In [ ]:
plt.plot(train_losses,label='train_loss')
plt.plot(test_losses,label='test_loss')
plt.legend()

Prediction

In [ ]:
# test_dir = r'/kaggle/working/test'
# from torchvision import transforms,models,datasets
# def transform_image(image):
#     image_path = r'{}'.format(image)
#     custom_image = torchvision.io.read_image(str(image_path)).type(torch.float32)
#     custom_image /= 255
#     custom_trans = transforms.Compose([transforms.Resize((224,224))])
#     custom_image_transformed = custom_trans(custom_image) 

#     return custom_image_transformed

# def predict_image(image,model):
#     model.eval()
#     with torch.no_grad():
#         custom_pred = model(image.unsqueeze(dim=0).to(device))
#         probs = torch.nn.functional.softmax(custom_pred[0],dim=0)
#         predicted_class = torch.argmax(probs).item()
#     return predicted_class

# predictions = []
# image_names = []
# ids = []
# for image in os.listdir(test_dir):
#     image_path = os.path.join(test_dir,image) #Full path
#     #adding name 
#     ids.append(image.split(".")[0])
#     image_names.append(image)
#     prediction = predict_image(transform_image(image_path),tl_model)
#     predictions.append(prediction)

    
# df = pd.DataFrame({"id":ids,"label":predictions})
# df.to_csv('submission.csv',index=False)

In [ ]:
test_dir = r'/kaggle/working/test'

In [ ]:
from torchvision import transforms,models,datasets

In [ ]:
def transform_image(image):
    image_path = r'{}'.format(image)
    custom_image = torchvision.io.read_image(str(image_path)).type(torch.float32)
    custom_image /= 255
    custom_trans = transforms.Compose([transforms.Resize((224,224))])
    custom_image_transformed = custom_trans(custom_image) 

    return custom_image_transformed

def predict_image(image,model):
    model.eval()
    with torch.no_grad():
        custom_pred = model(image.unsqueeze(dim=0).to(device))
        probs = torch.nn.functional.softmax(custom_pred[0],dim=0)
        predicted_class = torch.argmax(probs).item()
    return predicted_class




In [ ]:
predictions = []
image_names = []
ids = []
for image in os.listdir(test_dir):
    image_path = os.path.join(test_dir,image) #Full path
    #adding name 
    ids.append(image.split(".")[0])
    image_names.append(image)
    prediction = predict_image(transform_image(image_path),tl_model2)
    predictions.append(prediction)

In [ ]:
len(predictions),len(image_names)

In [ ]:
import pandas as pd

In [ ]:
os.chdir(r'/kaggle/working/')

In [ ]:
df = pd.DataFrame({"id":ids,"label":predictions})

In [ ]:
df.to_csv('submission.csv',index=False)